<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/model/cnnTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from google.colab import files

uploaded = files.upload()

Saving Model.py to Model.py


In [0]:
from Model import Model

In [0]:
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install -q torchsummary
    from torchsummary import summary


In [0]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [8]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

--2020-03-06 16:23:14--  https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4519 (4.4K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   4.41K  --.-KB/s    in 0s      

2020-03-06 16:23:14 (123 MB/s) - ‘utils.py’ saved [4519/4519]



In [9]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)


Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
Files already downloaded and verified


In [10]:
try: 
  del model
except:
  print('')
# Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
# Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
# The scecond one is the strided. The third: ConvPool. The last: All CNN.
model = Model()
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)

In [11]:
print(model)

Model(
  (model): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=False)
    (5): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Dropout(p=0.5, inplace=False)
    (9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (12): ReLU()
    (13): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
    (14): ReLU()
    (15): AdaptiveAvgPool2d(output_size=1)
    (16): Flatten()
    (17): Softmax(dim=None)
  )
  (conv_3_192_192_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_192_192): Conv2d(192, 192, kernel_size=(1, 1), stride=(1,

In [12]:
summary(model.model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Dropout-1            [-1, 3, 32, 32]               0
            Conv2d-2           [-1, 96, 30, 30]           7,296
              ReLU-3           [-1, 96, 30, 30]               0
         MaxPool2d-4           [-1, 96, 14, 14]               0
           Dropout-5           [-1, 96, 14, 14]               0
            Conv2d-6          [-1, 192, 12, 12]         460,992
              ReLU-7          [-1, 192, 12, 12]               0
         MaxPool2d-8            [-1, 192, 5, 5]               0
           Dropout-9            [-1, 192, 5, 5]               0
           Conv2d-10            [-1, 192, 5, 5]         331,968
             ReLU-11            [-1, 192, 5, 5]               0
           Conv2d-12            [-1, 192, 5, 5]          37,056
             ReLU-13            [-1, 192, 5, 5]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [0]:
def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best.pt")
        if gsync_save:
            gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [15]:
for epoch in range(350):
    scheduler.step()
    train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.301639
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.303400
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.302546
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.303373
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.303718
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.303277
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.302430
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.301419
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.302362
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.302124
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.301030
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.300812
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.302234
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.301587
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.299989
Train Epoch: 0 [48000/50000 (96%)]	Loss: 2.301575

Test set: Average loss: 0.0186, Accuracy: 1248/10000 (12%)



Uploading file best.pt: 100%|██████████| 100/100 [00:00<00:00, 109.02it/s]


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.305094
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.300279
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.296541
Train Epoch: 1 [9600/50000 (19%)]	Loss: 2.300409
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.291501
Train Epoch: 1 [16000/50000 (32%)]	Loss: 2.296857
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.287378
Train Epoch: 1 [22400/50000 (45%)]	Loss: 2.311122
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.315170
Train Epoch: 1 [28800/50000 (58%)]	Loss: 2.304002
Train Epoch: 1 [32000/50000 (64%)]	Loss: 2.283650
Train Epoch: 1 [35200/50000 (70%)]	Loss: 2.301900
Train Epoch: 1 [38400/50000 (77%)]	Loss: 2.309891
Train Epoch: 1 [41600/50000 (83%)]	Loss: 2.289171
Train Epoch: 1 [44800/50000 (90%)]	Loss: 2.308699
Train Epoch: 1 [48000/50000 (96%)]	Loss: 2.318862

Test set: Average loss: 0.0185, Accuracy: 1601/10000 (16%)



Uploading file best.pt: 100%|██████████| 100/100 [00:00<00:00, 102.05it/s]


...

Train Epoch: 186 [0/50000 (0%)]	Loss: 1.837283
Train Epoch: 186 [3200/50000 (6%)]	Loss: 1.869860
Train Epoch: 186 [6400/50000 (13%)]	Loss: 1.926592
Train Epoch: 186 [9600/50000 (19%)]	Loss: 1.815858
Train Epoch: 186 [12800/50000 (26%)]	Loss: 1.930065
Train Epoch: 186 [16000/50000 (32%)]	Loss: 1.950253
Train Epoch: 186 [19200/50000 (38%)]	Loss: 1.819105
Train Epoch: 186 [22400/50000 (45%)]	Loss: 1.888828
Train Epoch: 186 [25600/50000 (51%)]	Loss: 2.068895
Train Epoch: 186 [28800/50000 (58%)]	Loss: 1.878624
Train Epoch: 186 [32000/50000 (64%)]	Loss: 1.750628
Train Epoch: 186 [35200/50000 (70%)]	Loss: 1.859533
Train Epoch: 186 [38400/50000 (77%)]	Loss: 1.924182
Train Epoch: 186 [41600/50000 (83%)]	Loss: 1.935655
Train Epoch: 186 [44800/50000 (90%)]	Loss: 2.021167
Train Epoch: 186 [48000/50000 (96%)]	Loss: 1.786855

Test set: Average loss: 0.0154, Accuracy: 5600/10000 (56%)

Train Epoch: 187 [0/50000 (0%)]	Loss: 1.894843
Train Epoch: 187 [3200/50000 (6%)]	Loss: 1.815762
Train Epoch: 187 [6

Uploading file best.pt: 100%|██████████| 100/100 [00:00<00:00, 136.14it/s]


Train Epoch: 191 [0/50000 (0%)]	Loss: 1.870166
Train Epoch: 191 [3200/50000 (6%)]	Loss: 2.022897
Train Epoch: 191 [6400/50000 (13%)]	Loss: 1.782766
Train Epoch: 191 [9600/50000 (19%)]	Loss: 1.803323
Train Epoch: 191 [12800/50000 (26%)]	Loss: 1.955207
Train Epoch: 191 [16000/50000 (32%)]	Loss: 1.835431
Train Epoch: 191 [19200/50000 (38%)]	Loss: 1.892501
Train Epoch: 191 [22400/50000 (45%)]	Loss: 1.926761
Train Epoch: 191 [25600/50000 (51%)]	Loss: 1.772567
Train Epoch: 191 [28800/50000 (58%)]	Loss: 1.912673
Train Epoch: 191 [32000/50000 (64%)]	Loss: 1.764373
Train Epoch: 191 [35200/50000 (70%)]	Loss: 1.716042
Train Epoch: 191 [38400/50000 (77%)]	Loss: 2.059235
Train Epoch: 191 [41600/50000 (83%)]	Loss: 1.911463
Train Epoch: 191 [44800/50000 (90%)]	Loss: 1.937488
Train Epoch: 191 [48000/50000 (96%)]	Loss: 1.873920

Test set: Average loss: 0.0153, Accuracy: 5713/10000 (57%)

Train Epoch: 192 [0/50000 (0%)]	Loss: 1.940898
Train Epoch: 192 [3200/50000 (6%)]	Loss: 1.828376
Train Epoch: 192 [6

Uploading file best.pt: 100%|██████████| 100/100 [00:00<00:00, 123.92it/s]


Train Epoch: 195 [0/50000 (0%)]	Loss: 2.023034
Train Epoch: 195 [3200/50000 (6%)]	Loss: 1.952844
Train Epoch: 195 [6400/50000 (13%)]	Loss: 1.861782
Train Epoch: 195 [9600/50000 (19%)]	Loss: 1.915851
Train Epoch: 195 [12800/50000 (26%)]	Loss: 1.878592
Train Epoch: 195 [16000/50000 (32%)]	Loss: 1.922050
Train Epoch: 195 [19200/50000 (38%)]	Loss: 1.848041
Train Epoch: 195 [22400/50000 (45%)]	Loss: 1.724594
Train Epoch: 195 [25600/50000 (51%)]	Loss: 1.879107
Train Epoch: 195 [28800/50000 (58%)]	Loss: 1.835864
Train Epoch: 195 [32000/50000 (64%)]	Loss: 1.747580
Train Epoch: 195 [35200/50000 (70%)]	Loss: 1.971942
Train Epoch: 195 [38400/50000 (77%)]	Loss: 1.890983
Train Epoch: 195 [41600/50000 (83%)]	Loss: 1.900136
Train Epoch: 195 [44800/50000 (90%)]	Loss: 1.827035
Train Epoch: 195 [48000/50000 (96%)]	Loss: 1.839716

Test set: Average loss: 0.0153, Accuracy: 5757/10000 (58%)

Train Epoch: 196 [0/50000 (0%)]	Loss: 1.775117
Train Epoch: 196 [3200/50000 (6%)]	Loss: 1.794484
Train Epoch: 196 [6

In [0]:
# if in Google Colab, download your model with this
from google.colab import files
files.download("best.pt")